In [4]:
import os

import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

base_path = '/Users/amaxilatis/repositories/sparkworks/project-leeaf/dataset/olive_leaf'
### tomato leaf disease
TRAIN_PATH = f'{base_path}/train'
VAL_PATH = f'{base_path}/valid'
Test_path = f'{base_path}/test'

IMAGE_SIZE = [256, 256]
BATCH_SIZE = 32
NUM_CLASSES = 3
EPOCHS = 30


In [5]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=TRAIN_PATH,
    image_size=IMAGE_SIZE,
    shuffle=False,
    validation_split=0.2,
    subset="training"
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=VAL_PATH,
    image_size=IMAGE_SIZE,
    shuffle=False,
    validation_split=0.2,
    subset="validation"
)

Found 5990 files belonging to 3 classes.
Using 4792 files for training.
Found 980 files belonging to 3 classes.
Using 196 files for validation.


In [6]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=Test_path,
    image_size=IMAGE_SIZE,
    shuffle=False,

)

Found 970 files belonging to 3 classes.


In [7]:

checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'best_model2.h5')

In [8]:
class CustomModelCheckpointCallback(tf.keras.callbacks.ModelCheckpoint):

    def __init__(self, ignore_first=2, *args, **kwargs):
        super(CustomModelCheckpointCallback, self).__init__(*args, **kwargs)
        self.ignore_first = ignore_first

    def on_epoch_end(self, epoch, logs):
        if epoch + 1 > self.ignore_first:
            super().on_epoch_end(epoch, logs)

In [9]:
checkpoint_callback = CustomModelCheckpointCallback(ignore_first=10, filepath=checkpoint_path, monitor='val_loss',
                                                    mode='min', save_best_only=True, save_weights_only=False, verbose=1)

In [18]:

save_directory = 'transfer_models'
os.makedirs(save_directory, exist_ok=True)


def train_and_evaluate_models(model_name, train_dataset, test_dataset):
    # Load the pre-trained model
    model = tf.keras.applications.__getattribute__(model_name)(input_shape=IMAGE_SIZE + [3], weights='imagenet',
                                                               include_top=False)

    for layer in model.layers:
        layer.trainable = False

    x = tf.keras.layers.Flatten()(model.output)
    prediction = tf.keras.layers.Dense(10, activation='softmax')(x)

    existing_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

    existing_model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer="adam",
        metrics=['accuracy']
    )

    # Train the model
    existing_history = existing_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=40,
        callbacks=[checkpoint_callback],
        batch_size=32,
        shuffle=True
    )

    # Calculate precision, recall, and F1 score
    y_pred = existing_model.predict(test_dataset)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true = np.concatenate([y for x, y in test_dataset], axis=0)

    # Calculate accuracy, precision, recall, and F1 score with zero_division parameter
    accuracy = accuracy_score(y_true, y_pred_labels)
    precision = precision_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    f1 = f1_score(y_true, y_pred_labels, average='weighted', zero_division=1)

    model_save_path = os.path.join(save_directory, f'{model_name}.h5')
    existing_model.save(model_save_path)

    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

In [22]:
# Define the list of model names to train and evaluate
model_names = ['ResNet50', 'VGG19', 'VGG16', 'InceptionV3', 'DenseNet121', 'MobileNetV2']
evaluation_results = {}

In [23]:
model_name = model_names[0]
evaluation_results[model_name] = train_and_evaluate_models(model_name, train_dataset, test_dataset)

Epoch 1/40
150/150 [==============================] - 126s 833ms/step - loss: 36.8757 - accuracy: 0.9034 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/40
150/150 [==============================] - 124s 824ms/step - loss: 175.6618 - accuracy: 0.7611 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/40
150/150 [==============================] - 122s 817ms/step - loss: 57.2361 - accuracy: 0.8362 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/40
150/150 [==============================] - 263s 2s/step - loss: 25.4225 - accuracy: 0.8888 - val_loss: 0.0213 - val_accuracy: 0.9949
Epoch 5/40
150/150 [==============================] - 124s 826ms/step - loss: 17.5048 - accuracy: 0.9061 - val_loss: 0.0988 - val_accuracy: 0.9949
Epoch 6/40
150/150 [==============================] - 124s 826ms/step - loss: 13.7808 - accuracy: 0.9167 - val_loss: 0.2729 - val_accuracy: 0.9949
Epoch 7/40
150/150 [==============================] - 124s 826ms/step - loss: 10.4716 - accuracy: 0.9282 - v

In [24]:
model_name = model_names[1]
evaluation_results[model_name] = train_and_evaluate_models(model_name, train_dataset, test_dataset)

80134624/80134624 [==============================] - 4s 0us/step
Epoch 1/40
150/150 [==============================] - 493s 3s/step - loss: 31.2342 - accuracy: 0.9071 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/40
150/150 [==============================] - ETA: 0s - loss: 149.4347 - accuracy: 0.7690

KeyboardInterrupt: 

In [ ]:
model_name = model_names[2]
evaluation_results[model_name] = train_and_evaluate_models(model_name, train_dataset, test_dataset)

In [ ]:
model_name = model_names[3]
evaluation_results[model_name] = train_and_evaluate_models(model_name, train_dataset, test_dataset)

In [ ]:
model_name = model_names[4]
evaluation_results[model_name] = train_and_evaluate_models(model_name, train_dataset, test_dataset)

In [ ]:
model_name = model_names[5]
evaluation_results[model_name] = train_and_evaluate_models(model_name, train_dataset, test_dataset)

In [ ]:
# Save the evaluation results to a text file
with open('evaluation_results.txt', 'w') as f:
    for model_name, metrics in evaluation_results.items():
        f.write(f'{model_name}\n')
        for metric, score in metrics.items():
            f.write(f'{metric}: {score}\n')
        f.write('\n')